## Day 2 - Rock Paper Scissors

>The Elves begin to set up camp on the beach. To decide whose tent gets to be closest to the snack storage, a giant Rock Paper Scissors tournament is already in progress.
>
>Rock Paper Scissors is a game between two players. Each game contains many rounds; in each round, the players each simultaneously choose one of Rock, Paper, or Scissors using a hand shape. Then, a winner for that round is selected: Rock defeats Scissors, Scissors defeats Paper, and Paper defeats Rock. If both players choose the same shape, the round instead ends in a draw.
>
>Appreciative of your help yesterday, one Elf gives you an encrypted strategy guide (your puzzle input) that they say will be sure to help you win. "The first column is what your opponent is going to play: A for Rock, B for Paper, and C for Scissors. The second column--" Suddenly, the Elf is called away to help with someone's tent.
>
>The second column, you reason, must be what you should play in response: X for Rock, Y for Paper, and Z for Scissors. Winning every time would be suspicious, so the responses must have been carefully chosen.
>
>The winner of the whole tournament is the player with the highest score. Your total score is the sum of your scores for each round. The score for a single round is the score for the shape you selected (1 for Rock, 2 for Paper, and 3 for Scissors) plus the score for the outcome of the round (0 if you lost, 3 if the round was a draw, and 6 if you won).
>
>Since you can't be sure if the Elf is trying to help you or trick you, you should calculate the score you would get if you were to follow the strategy guide.
>
>For example, suppose you were given the following strategy guide:
>```
>A Y
>B X
>C Z
>```
>This strategy guide predicts and recommends the following:
>
>- In the first round, your opponent will choose Rock (A), and you should choose Paper (Y). This ends in a win for you with a score of 8 (2 because you chose Paper + 6 because you won).
>- In the second round, your opponent will choose Paper (B), and you should choose Rock (X). This ends in a loss for you with a score of 1 (1 + 0).
>- The third round is a draw with both players choosing Scissors, giving you a score of 3 + 3 = 6.
>
>In this example, if you were to follow the strategy guide, you would get a total score of 15 (8 + 1 + 6).

## Read input

For reading input today, I'll start by defining some data structures.

First one will be `RPSScore` (short for Rock-Paper-Scissors Score) which is an `IntEnum`. [Enums](https://docs.python.org/3/library/enum.html) are great when you want to give descriptive names to individual values and by using subclass `IntEnum`, we can do maths with the results directly with the enums.

Here I decided to give each Enum member the value that is its score in the final calculation. That way, when calculating the score, I can calculate win/draw/loss score + the enum member and get the final score.

Second data structure is another `IntEnum`, this time for the score for match output. Loss gains the player 0, draw 3 and win 6 points.

Third data structure is `input_to_RPS` which maps each input (A, B, C, X, Y, Z) to its corresponding Enum member.

In [50]:
from enum import IntEnum

from utils import read_input


class RPSScore(IntEnum):
    ROCK = 1
    PAPER = 2
    SCISSORS = 3
    
class MatchScore(IntEnum):
    LOSS = 0
    DRAW = 3
    WIN = 6
    
input_to_RPS = {
    "A": RPSScore.ROCK,
    "B": RPSScore.PAPER,
    "C": RPSScore.SCISSORS,
    "X": RPSScore.ROCK,
    "Y": RPSScore.PAPER,
    "Z": RPSScore.SCISSORS
}

def transformer(line):
    opponent, player = line.split(' ')
    return [input_to_RPS[player], input_to_RPS[opponent]]

example_strategy = read_input(2, transformer, True)
strategy = read_input(2, transformer)

## Calculating the score

Last summer I wrote [a blog post about writing unit tests in Jupyter Notebooks](https://hamatti.org/posts/unit-test-your-python-code-in-jupyter-notebooks/) so when I was building this scoring function, I wanted to make sure I calculated all the cases correctly and decided to use [doctest](https://docs.python.org/3/library/doctest.html).

In my blog post, I said that doctest is good for simple functions but gets out of hand real fast. I think this is a perfect example of that. With just 9 different cases, the docstring becomes 20 lines long and not that easy to read and follow. But it works and having tests made me not run into harder-to-debug issues down the line.

The scoring uses [Python's pattern matching, available in Python since 3.10](https://hamatti.org/posts/pattern-matching-is-coming-to-python/) and a few nice features of it:

1. For the first clause, it uses a guard in the form of a if statement that matches to any two inputs as long as they are the same.
2. For the second clause, it uses the "or" functionality, matching to any of the cases where player wins
3. It uses the default case for anything else

And a nice benefit of defining our scores in enums earlier, we don't see a single number in this function. That means we don't need to know the exact values in this function, leading to less errors if they would change in the future for example.

In this case, as this function is the only one today that has doctests, I'm running the test suite immediately in the same code block but it's important to note that it will run all the doctests in the file/notebook so sometimes it makes sense to run as its own block or at the end of the file.

In [60]:
def score_round(player, opponent):
    """
    >>> score_round(RPSScore.ROCK, RPSScore.ROCK)
    4
    >>> score_round(RPSScore.PAPER, RPSScore.PAPER)
    5
    >>> score_round(RPSScore.SCISSORS, RPSScore.SCISSORS)
    6
    >>> score_round(RPSScore.ROCK, RPSScore.PAPER)
    1
    >>> score_round(RPSScore.ROCK, RPSScore.SCISSORS)
    7
    >>> score_round(RPSScore.PAPER, RPSScore.ROCK)
    8
    >>> score_round(RPSScore.PAPER, RPSScore.SCISSORS)
    2
    >>> score_round(RPSScore.SCISSORS, RPSScore.ROCK)
    3
    >>> score_round(RPSScore.SCISSORS, RPSScore.PAPER)
    9
    """
    match [player, opponent]:
        case [player, opponent] if player == opponent:
            return MatchScore.DRAW + player
        case ([RPSScore.ROCK, RPSScore.SCISSORS] |
              [RPSScore.PAPER, RPSScore.ROCK] | 
              [RPSScore.SCISSORS, RPSScore.PAPER]):
            return MatchScore.WIN + player
        case _:
            return MatchScore.LOSS + player
        
import doctest

doctest.testmod()

TestResults(failed=0, attempted=9)

## Part 1

> **What would your total score be if everything goes exactly according to your strategy guide?**

To calculate the total score, we run all the plays from the strategy and pass them to `score_round` using the `*` operator that unpacks the elements of an iterator, in this case a list of two, into separate arguments for the function.

In [46]:
score = sum(score_round(*plays) for plays in strategy)
print(f'Part 1: {score}')
assert score == 12740

Part 1: 12740


## Part 2

I think this might be the first time in my Advent of Code solving era where the second part actually changes the input reading instead of just the solution.

Now, instead of parsing two plays, we get a play and a result and using the previous enums and scoring function, need to add a bit to figure out the right play.

`playresult_to_choice` is a horrible variable name but I'll live with it.

In [58]:
playresult_to_choice = {
    ('A', 'X'): RPSScore.SCISSORS,
    ('A', 'Y'): RPSScore.ROCK,
    ('A', 'Z'): RPSScore.PAPER,
    
    ('B', 'X'): RPSScore.ROCK,
    ('B', 'Y'): RPSScore.PAPER,
    ('B', 'Z'): RPSScore.SCISSORS,
    
    ('C', 'X'): RPSScore.PAPER,
    ('C', 'Y'): RPSScore.SCISSORS,
    ('C', 'Z'): RPSScore.ROCK
}

def part2_transformer(line):
    opponent, result = line.split(' ')
    player = playresult_to_choice[opponent, result]
    return player, input_to_RPS[opponent]
    

part2_example_strategy = read_input(2, part2_transformer, True)
part2_strategy = read_input(2, part2_transformer)

The actual scoring calculation is exactly the same as in part 1 as the only change needed to be made was into how we parse the strategy guide.

In [59]:
part2_score = sum(score_round(*plays) for plays in part2_strategy)
print(f'Part 2: {part2_score}')
assert part2_score == 11980

Part 2: 11980
